<a href="https://colab.research.google.com/github/sujitpal/nlp-deeplearning-ai-examples/blob/master/lf2_longformer_sentiment_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LongTransformer Tests (Fine tuning)

In this notebook, we will fine tune a `LongformerForSequenceClassification` model to do sentiment analysis. Our dataset will come from the [Sentiment Labeled dataset from the UCI ML repository](https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences).

### Machine Setup

In [1]:
!pip install transformers

In [2]:
%%bash
(
if [ -n "sentiment labelled sentences.zip" ]
then
  wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip"
  unzip -a "sentiment labelled sentences.zip"
fi
)

Archive:  sentiment labelled sentences.zip


--2020-09-15 23:00:11--  https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84188 (82K) [application/x-httpd-php]
Saving to: ‘sentiment labelled sentences.zip.1’

     0K .......... .......... .......... .......... .......... 60%  865K 0s
    50K .......... .......... .......... ..                   100% 77.6M=0.06s

2020-09-15 23:00:11 (1.38 MB/s) - ‘sentiment labelled sentences.zip.1’ saved [84188/84188]

replace sentiment labelled sentences/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


### Imports

In [3]:
import numpy as np
import os

import torch
import torch.nn as nn
import torch.nn.functional as F

# from torch.nn import functional as F
from transformers import (
    LongformerTokenizer, LongformerForSequenceClassification,
    AdamW
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

### Data Preparation

In [4]:
def load_sentences_and_labels(folder):
  sentences, labels = [], []
  for labelled_file in os.listdir(folder):
    if labelled_file.split('.')[0].endswith("_labelled"):
      flab = open(os.path.join(folder, labelled_file), "r")
      for line in flab:
        sentence, label = line.strip().split('\t')
        sentences.append(sentence)
        labels.append(label)
      flab.close()
  return sentences, labels

sentences, labels = load_sentences_and_labels("sentiment labelled sentences")
len(sentences), len(labels)

(3000, 3000)

In [5]:
sentences_train, sentences_test, labels_train, labels_test = train_test_split(
    sentences, labels, test_size=0.2)

len(sentences_train), len(labels_train), len(sentences_test), len(labels_test)

(2400, 2400, 600, 600)

### Define and Train Model

In [6]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
dev

device(type='cuda')

In [7]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

model = LongformerForSequenceClassification.from_pretrained(
    "allenai/longformer-base-4096", return_dict=True)

# freeze the encoder portion, we just want to update the head weights
for param in model.base_model.parameters():
    param.requires_grad = False

_ = model.to(dev)
_ = model.train()

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', '

In [8]:
optimizer = AdamW(model.parameters(), lr=1e-5)

In [9]:
NUM_EPOCHS = 2
BATCH_SIZE = 32

In [10]:
num_train_sents = len(sentences_train)
losses = []
for e in range(NUM_EPOCHS):

  # shuffle the dataset
  permutation = np.random.permutation(len(sentences_train))

  for i in range(0, num_train_sents, BATCH_SIZE):

    optimizer.zero_grad()

    indices = set(permutation[i : i + BATCH_SIZE].tolist())
    sentences_batch = [s for i, s in enumerate(sentences_train) if i in indices]
    labels_batch = [int(lb) for i, lb in enumerate(labels_train) if i in indices]

    encoding = tokenizer(sentences_batch, return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(dev)
    attention_mask = encoding['attention_mask'].to(dev)
    labels = torch.tensor(labels_batch).unsqueeze(0).to(dev)

    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss

    loss_numpy = loss.item()
    losses.append(loss_numpy)
    
    loss.backward()
    optimizer.step()

  print("EPOCH {:d} | LOSS {:.3f}".format(e, loss_numpy))


EPOCH 0 | LOSS 0.696
EPOCH 1 | LOSS 0.693


### Predict using Trained Model

In [11]:
model.eval()

num_test_sents = len(sentences_test)
predictions = []
for i in range(0, num_test_sents, BATCH_SIZE):
  sentences_batch = sentences_test[i : i + BATCH_SIZE]
  encoding = tokenizer(sentences_batch, return_tensors='pt', padding=True, truncation=True)
  input_ids = encoding['input_ids'].to(dev)
  attention_mask = encoding['attention_mask'].to(dev)

  preds_batch = model(input_ids, attention_mask=attention_mask)

  predictions.extend([0 if lg[0] > lg[1] else 1 
    for lg in torch.exp(preds_batch.logits).detach().cpu().numpy().tolist()])


### Evaluate Model

In [12]:
labels = [int(lb) for lb in labels_test]
print("accuracy: {:.3f}".format(accuracy_score(labels, predictions)))
print("confusion matrix")
print(confusion_matrix(labels, predictions))

accuracy: 0.727
confusion matrix
[[240  63]
 [101 196]]
